# 필요 구성 라이브러리 다운로드

In [1]:
# !pip install --upgrade pip
# !pip install konlpy
# !pip install nltk
# !pip install sklearn
# !pip install pandas
# !pip install jellyfish
# !pip install segtok
# !pip install networkx
# !pip install matplotlib
# !pip install tensorflow
# !pip install krwordrank
# !pip install soynlp
# !pip install textrank
# !pip install sentence_transformers
# !pip install spaCy
# !python -m spacy download ko_core_news_sm

# 텍스트 마이닝 실험

In [1]:
# 필요 라이브러리 선언
from pprint import pprint

from yake import korea_token
from yake.korea_token import edit_josa, edit_sentences
import math

from konlpy.tag import Komoran
from collections import Counter

import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from math import log
import numpy as np
from collections import defaultdict

import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.request
np.random.seed(seed=0)

from krwordrank.word import KRWordRank
from krwordrank.sentence import summarize_with_sentences
from krwordrank.word import summarize_with_keywords

from scipy.sparse import csr_matrix
from soynlp.tokenizer import MaxScoreTokenizer
from sklearn.metrics import pairwise_distances

import networkx

# from textrank import KeywordSummarizer

import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

import spacy
from spacy.lang.ko.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

## 현재 Kyake 모델 

In [7]:
from yake import yake
from yake.datarepresentation_korea import DataCore
import pandas as pd

#text = "Don't be fooled by the dark fool sounding name. Mr. Jone's Orphange is as cheery goes for a pastry shop. My dogs sounded is cute and My dog sound is not cute" 
#text = "hello world i am a boy. I want to go ride a car. dog is crazy and crazy dog barking"
#text =  "Google is acquiring data science community Kaggle. Sources tell us that Google is acquiring Kaggle, a platform that hosts data science and machine learning competitions. Details about the transaction remain somewhat vague , but given that Google is hosting its Cloud Next conference in San Francisco this week, the official announcement could come as early as tomorrow. Reached by phone, Kaggle co-founder CEO Anthony Goldbloom declined to deny that the acquisition is happening. Google itself declined 'to comment on rumors'. Kaggle, which has about half a million data scientists on its platform, was founded by Goldbloom and Ben Hamner in 2010. The service got an early start and even though it has a few competitors like DrivenData, TopCoder and HackerRank, it has managed to stay well ahead of them by focusing on its specific niche. The service is basically the de facto home for running data science and machine learning competitions. With Kaggle, Google is buying one of the largest and most active communities for data scientists - and with that, it will get increased mindshare in this community, too (though it already has plenty of that thanks to Tensorflow and other projects). Kaggle has a bit of a history with Google, too, but that's pretty recent. Earlier this month, Google and Kaggle teamed up to host a $100,000 machine learning competition around classifying YouTube videos. That competition had some deep integrations with the Google Cloud Platform, too. Our understanding is that Google will keep the service running - likely under its current name. While the acquisition is probably more about Kaggle's community than technology, Kaggle did build some interesting tools for hosting its competition and 'kernels', too. On Kaggle, kernels are basically the source code for analyzing data sets and developers can share this code on the platform (the company previously called them 'scripts'). Like similar competition-centric sites, Kaggle also runs a job board, too. It's unclear what Google will do with that part of the service. According to Crunchbase, Kaggle raised $12.5 million (though PitchBook says it's $12.75) since its launch in 2010. Investors in Kaggle include Index Ventures, SV Angel, Max Levchin, Naval Ravikant, Google chief economist Hal Varian, Khosla Ventures and Yuri Milner"
#text = "러시아가 우크라이나 키이우 등을 공습한 것은 군 내부 비판과 블라디미르 푸틴 러시아 대통령의 자존심 때문이라는 분석이 나왔다. 영국 가디언지는 10일(현지시간) 푸틴 대통령의 대규모 공습은 국내 군 비판세력, 러시아가 침공에서 실패하고 있다는 사실, 크림대교 폭발 후 상처받은 자존심에 대한 절박한 답변이라고 풀이했다. 가디언에 따르면 미국 싱크탱크 카네기국제평화재단의 선임 연구원 안드레이 콜레스니코프는 '지금 푸틴이 하는 것은 사소한 복수'라며 '개인적 복수도 있다'고 말했다. 러시아 전쟁 전문가와 군사 블로거 등은 수개월간 우크라이나를 상대로 전면전을 벌이라고 촉구해왔고 키이우 등의 거리에 시신이 있는 끔찍한 사진이 나오자 지금은 만족해한다. 최근 러시아군 수뇌부를 비판했던 람잔 카디로프 체첸 공화국 수장은 '(볼로디미르) 젤렌스키(우크라이나 대통령), 우리는 러시아가 아직 제대로 시작도 안했다고 경고했다'라며 '이제 전쟁 진행에 100% 만족한다'고 말했다. 푸틴 대통령은 이날 러시아 안전보장이사회 회의를 주재하고 크림대교 폭발의 배후로 우크라이나 정보 당국을 겨냥하면서 '이런 종류의 범죄에 대응을 하지 않는 것은 불가능하다'고 말했다. 그러나 드미트로 쿨레바 우크라이나 외무장관은 '러시아는 크림대교 사건 전에도 우크라이나에 계속 미사일 공격을 했다'며 '푸틴은 전투 패배로 절박한 상황이며 전황을 유리하게 바꾸려고 미사일 공포를 사용한다'고 반박했다. 우크라이나 구조대원들이 10일(현지시간) 수도 키이우 시내에서 러시아군의 미사일 공격을 당한 현장을 조사하고 있다. 이날 오전에 키이우 시내에 여러차례 폭발이 발생했다. 푸틴 대통령은 이번 공격이 국방부의 요청에 따라 이뤄졌다고 주장했는데, 이것이 사실이라면 이는 새로운 합동군 총사령관 세르게이 수로비킨의 첫번째 결정이라고 가디언은 전했다. 수로비킨 사령관과 함께 일했던 전 국방부 관계자는 가디언지에 '오늘 키이우에서 벌어진 일이 놀랍지 않았다. 그는 매우 무자비하고 사람 목숨을 신경 쓰지 않는다'며 '그의 손이 우크라이나인의 피로 뒤덮일까 걱정된다'고 말했다. 그러나 푸틴 대통령이 이날 공습으로 얻은 매파들의 호평은 오래가지 않을 수 있다. 그는 이번과 같은 대규모 미사일 공격은 러시아 영토 공격 시 대응으로 남겨둘 것이라고 말했는데 강경파들은 전면전을 원하고 있다. 정치 평론가인 세르게이 마르코프는 '러시아 여론은 대규모 공격과 우크라이나군이 사용할 가능성이 있는 인프라 완전파괴를 원한다'고 말했다. 한편으론 비판 세력의 목소리는 그다지 의미가 없을 수도 있어 보인다. 전쟁과 관련한 의사결정 과정은 여전히 불투명하다. 콜레스니코프 선임 연구원은 '푸틴 대통령으로선 매파와 극보수파의 불만에 대응하는 것이 중요하겠지만 나는 그들의 영향력을 과장하진 않을 것'이라며 '푸틴 자신이 가장 매파적이고 극보수적인 인물'이라고 말했다. 러시아에서 나오는 한 가지 이론은 푸틴 대통령이 악명 높은 새로운 군사령관을 임명함으로써 전쟁에서 국방부의 성과에 대한 분노를 줄이려고 한다는 것이다. 수로비킨 사령관과 2020년까지 함께 일했던 전 공군 중위인 글렙 이리소프는 '수로비킨은 강경파들을 선호하고 와그너 용병회사와도 좋은 관계를 유지한다'며 '그러나 그가 매우 잔인한 동시에 유능한 사령관이지만 모든 문제를 풀 수는 없을 것'이라고 말했다."
#text = "러시아가 우크라이나 키이우 등을 공습한 것은 군 내부 비판과 블라디미르 푸틴 러시아 대통령의 자존심 때문이라는 분석이 나왔다. 영국 가디언지는 10일(현지시간) 푸틴 대통령의 대규모 공습은 국내 군 비판세력, 러시아가 침공에서 실패하고 있다는 사실, 크림대교 폭발 후 상처받은 자존심에 대한 절박한 답변이라고 풀이했다. 가디언에 따르면 미국 싱크탱크 카네기국제평화재단의 선임 연구원 안드레이 콜레스니코프는 '지금 푸틴이 하는 것은 사소한 복수'라며 '개인적 복수도 있다'고 말했다. 러시아 전쟁 전문가와 군사 블로거 등은 수개월간 우크라이나를 상대로 전면전을 벌이라고 촉구해왔고 키이우 등의 거리에 시신이 있는 끔찍한 사진이 나오자 지금은 만족해한다. 최근 러시아군 수뇌부를 비판했던 람잔 카디로프 체첸 공화국 수장은 '(볼로디미르) 젤렌스키(우크라이나 대통령), 우리는 러시아가 아직 제대로 시작도 안했다고 경고했다'라며 '이제 전쟁 진행에 100% 만족한다'고 말했다. 푸틴 대통령은 이날 러시아 안전보장이사회 회의를 주재하고 크림대교 폭발의 배후로 우크라이나 정보 당국을 겨냥하면서 '이런 종류의 범죄에 대응을 하지 않는 것은 불가능하다'고 말했다. 그러나 드미트로 쿨레바 우크라이나 외무장관은 '러시아는 크림대교 사건 전에도 우크라이나에 계속 미사일 공격을 했다'며 '푸틴은 전투 패배로 절박한 상황이며 전황을 유리하게 바꾸려고 미사일 공포를 사용한다'고 반박했다. 우크라이나 구조대원들이 10일(현지시간) 수도 키이우 시내에서 러시아군의 미사일 공격을 당한 현장을 조사하고 있다. 이날 오전에 키이우 시내에 여러차례 폭발이 발생했다. 푸틴 대통령은 이번 공격이 국방부의 요청에 따라 이뤄졌다고 주장했는데, 이것이 사실이라면 이는 새로운 합동군 총사령관 세르게이 수로비킨의 첫번째 결정이라고 가디언은 전했다. 수로비킨 사령관과 함께 일했던 전 국방부 관계자는 가디언지에 '오늘 키이우에서 벌어진 일이 놀랍지 않았다. 그는 매우 무자비하고 사람 목숨을 신경 쓰지 않는다'며 '그의 손이 우크라이나인의 피로 뒤덮일까 걱정된다'고 말했다. 그러나 푸틴 대통령이 이날 공습으로 얻은 매파들의 호평은 오래가지 않을 수 있다. 그는 이번과 같은 대규모 미사일 공격은 러시아 영토 공격 시 대응으로 남겨둘 것이라고 말했는데 강경파들은 전면전을 원하고 있다. 정치 평론가인 세르게이 마르코프는 '러시아 여론은 대규모 공격과 우크라이나군이 사용할 가능성이 있는 인프라 완전파괴를 원한다'고 말했다. 한편으론 비판 세력의 목소리는 그다지 의미가 없을 수도 있어 보인다. 전쟁과 관련한 의사결정 과정은 여전히 불투명하다. 콜레스니코프 선임 연구원은 '푸틴 대통령으로선 매파와 극보수파의 불만에 대응하는 것이 중요하겠지만 나는 그들의 영향력을 과장하진 않을 것'이라며 '푸틴 자신이 가장 매파적이고 극보수적인 인물'이라고 말했다. 러시아에서 나오는 한 가지 이론은 푸틴 대통령이 악명 높은 새로운 군사령관을 임명함으로써 전쟁에서 국방부의 성과에 대한 분노를 줄이려고 한다는 것이다. 수로비킨 사령관과 2020년까지 함께 일했던 전 공군 중위인 글렙 이리소프는 '수로비킨은 강경파들을 선호하고 와그너 용병회사와도 좋은 관계를 유지한다'며 '그러나 그가 매우 잔인한 동시에 유능한 사령관이지만 모든 문제를 풀 수는 없을 것'이라고 말했다."
#text = "키릴로 티모셴코 대통령실 차장은 텔레그램을 통해 중대 기반 시설들에 자폭 드론을 동원한 또 다른 공격이 있었다고 말했다. 그는 공격을 받은 기반 시설이 어디인지는 언급하지 않았다.우크라이나 당국에 따르면, 우크라이나에서는 최근 몇주째 이란제 샤헤드-136 드론을 이용한 러시아군의 공격이 이어지고 있다. 이란은 러시아에 자국산 드론을 공급했다는 것을 부인하고 있다."
#text = "현대차에 이어 기아도 올해 3분기(7~9월)에 사상 최대 분기 매출을 기록했다. 그러나 세타 GDI 엔진 리콜 관련 1조5442억원의 대규모 충당금 반영으로 기아의 3분기 영업이익은 전년 동기 대비 42.1% 감소한 7682억원에 그쳤다. 다만 일회성 비용인 리콜 충당금을 제외하면 사실상 영업이익이 2조원을 넘어서는 양호한 흐름을 이어갔다. ◇3분기 매출은 역대 최대…1.5조 충당금 반영 영업이익은 42%↓ 기아는 25일 '2022년 3분기 경영실적 컨퍼런스 콜'을 통해 연결 기준 매출액이 23조1616억원으로 전년 동기 대비 30.5% 늘었다고 밝혔다. 이는 지난 2분기에 기록한 역대 최대 분기 매출 21조8760억원을 뛰어넘은 것이다. 영업이익은 7682억원으로 전년 동기 대비 42.1% 줄었다. 지난 2분기와 비교하면 65.6%나 감소했다. 지난 2분기에는 2조2340억원으로 역대 최대 분기 영업이익을 기록했었다. 판매대수 증가와 고부가가치 차종 중심의 믹스 개선, 우호적 환율 효과에도 불구하고 세타 GDi 엔진 관련 충당금 1조5442억원이 반영돼 영업이익이 1조원대 밑으로 떨어졌다. 영업이익률도 3.3%로 4.2%p 떨어졌다. 3분기 경상이익은 7300억원, 당기순이익은 4589억원을 기록했다. 기아 관계자는 부품 수급 개선으로 판매가 증가했고, 높은 상품 경쟁력을 바탕으로 한 고수익 차량 중심의 판매로 수익 구조 개선이 지속된 가운데 우호적 환율 영향도 강하게 작용했으나, 엔진 품질비용이 크게 반영된 결과 영업이익 감소를 피할 수 없었다고 밝혔다. 이어 다만 전 차종과 전 지역에 걸친 강한 수요는 지속되고 있는 만큼, 4분기에는 반도체 등 부품 수급 개선과 연계한 공급 확대를 통해 판매 회복과 수익성 강화가 이어질 것으로 기대한다고 덧붙였다. ◇3분기 글로벌 판매 75만2104대 판매…전년比 9.9%↑ 기아는 3분기 글로벌 시장에서 총 75만2104대를 판매했다고 밝혔다. 글로벌 차량용 반도체 및 기타 부품의 공급난 완화로 전년 동기 대비 9.9% 늘었다. 국내 시장에서는 전년 동기 대비 6.2% 증가한 13만 2768대를 판매했다. 이는 EV6·신형 니로 등 신차 효과와 반도체 공급난 완화 등으로 SUV 주요 모델들의 대기 수요가 일부 해소된 따른 것이다. 해외 시장에서는 10.7% 증가한 61만9336대를 판매했다. 러시아-우크라이나 전쟁으로 러시아 판매 중단의 영향이 본격화됐지만 수익성이 높은 타 지역으로의 물량 전환과 인도공장의 3교대 전환, 카렌스와 신형 스포티지 등 신차 효과로 대부분 지역에서 판매가 증가했다. 기아의 3분기 친환경차 판매는 전년 동기 대비 46.8% 증가한 12만3000대를 기록했다. 유형별로 △전기차가 4만대(전년 대비 34.3%↑) △하이브리드가 6만2000대(67.0%↑) △플러그인 하이브리드가 2만1000대(24.1%↑)였다. 전체 판매 차종 중 친환경차 비중도 전년 대비 5.6%p 오른 16.8%에 달했다. 국내는 12.3%, 서유럽은 11.%, 미국은 3.2%를 기록하는 등 전년 동기 대비 전기차 판매 비중이 크게 올랐다. 전기차 판매가 특정 지역에 집중되던 것에서 벗어나 시장도 다변화됐다. 지난해 3분기 서유럽 비중이 52.9%에 달했으나 올해에는 국내가 40.8%로 가장 높았고 서유럽 38.9%, 미국 14.6% 등 주요 시장에서 고른 판매를 기록했다. ◇국제 정세·경제 상황 불안하지만…4분기 가시적 실적 개선 기대 기아는 국제 정세 불안에 따른 원자재 가격 변동성 심화, 고금리와 인플레이션으로 인한 구매 심리 위축 등 불안정한 대외 환경을 예의주시하면서도 4분기 가시적인 실적 개선을 기대했다. 반도체 등 부품 수급 상황 개선과 연계해 공급을 최대한 늘림으로써 높은 대기 수요를 빠르게 해소하고, 친환경차와 고수익 RV 모델 중심으로 판매를 확대해 수익성 강화를 지속 추진한다는 계획이다. 전 세계에서 인정받고 있는 전기차 EV6의 생산·판매를 확대하면서 미국에서 텔루라이드 상품성 개선 모델 및 신형 스포티지, 유럽에서 신형 니로, 인도에서 카렌스 등 시장별 핵심 차종 판매 본격화에 집중한다는 방침이다. 아울러 제품 및 트림 믹스를 지속적으로 높이고, 개선된 브랜드 가치와 상품성에 부응하는 가격 정책을 이어가며 수익성도 극대화할 예정이다."
text = "이마트가 인플레이션 방어를 명분으로 야심 차게 실행했던 연말까지 생활필수품 최저가 책정 전략을 최근 중단했다. 이상기후발(發) 신선식품 가격급등, 고환율로 인한 수입 물가 상승 등의 벽을 끝내 넘지 못한 것으로 풀이된다. e커머스의 부상 이후 이마트가 과거와 같이 유통업계에서 강력한 영향력을 행사하기 어려운 여건이 조성된 것도 이런 결정을 내린 요인 중 하나로 꼽힌다. 벽에 부딪힌 최저가 마케팅 25일 유통업계에 따르면 이마트는 이달 들어 40대 생필품에 대한 업계 최저가 판매를 종료했다. 지난 7월 계란, 쌀, 우유, 휴지, 칫솔 등 40대 상품의 가격을 매일 모니터링해 최저가로 팔겠다고 선언한 지 3개월 만이다. 당시 이마트는 최저가 판매 대상 품목을 연말까지 확대해나가고, 고물가 상황이 진정되지 않으면 기간을 더 연장하겠다고 강조했다. 강희석 사장은 이마트에 가면 생필품을 가장 싸게 살 수 있다는 메시지를 확실하게 전하겠다고 했다. 하지만 글로벌 인플레의 파고가 이마트가 감당할 수 있는 수준을 넘어서면서 상황이 달라졌다. 최근엔 원·달러 환율까지 급등해 그나마 가격이 저렴했던 수입식품 가격까지 뛰었다. 그만큼 이마트의 부담은 더 커졌다. 이마트 관계자는 환율 급등과 원재료 가격 인상 등으로 주요 납품업체들의 부담이 커지는 실상을 감안해 최저가 판매를 부득이하게 중단했다고 설명했다. 대신 자체브랜드(PB) 상품 가격을 동결해 소비자들의 부담을 덜어주겠다는 방침이다. 납품업체들 반발 거세 유통업계에선 이마트가 최저가 정책을 종료한 데엔 유통산업의 환경이 과거와 같지 않다는 점도 영향을 미쳤을 것이란 분석이 나온다. 이마트가 최저가를 유지하려면 협력업체로부터 납품받는 단가를 낮추거나, 적정 마진을 포기하고 상품을 팔아야 한다. 이런 가운데 e커머스가 유통업의 새 강자로 부상하면서 납품업체들이 이마트에 휘둘리지 않을 수 있는 여건이 조성됐다. 한 중소 식품사 관계자는 이미 시장에선 최저가라고 불리는 가격의 수준이 대략 정해져 있는 가운데 각 유통업체가 준비하는 특별 행사에 따라 납품사들이 가격을 더 낮추는 구조라며 이마트를 절대 무시할 수 없지만, 이마트의 무조건 최저가를 굳이 따르지 않더라도 납품할 수 있는 유통업체들은 많다고 말했다. 유명무실해진 최저가 경쟁 최저가 경쟁이 유명무실해진 것도 조기 종료의 배경으로 지목된다. 이마트는 그간 매일 경쟁사의 가격을 확인하고, 주요 상품 가격을 최저가로 조정했지만, 경쟁사와의 가격 차이는 대부분 10~20원에 불과했다. 한 e커머스업체 식품 담당 바이어는 소비자들이 10원이라도 더 싸게 사기 위해 차를 끌고 대형마트를 돌아다니는 시대는 지났다며 e커머스 기업들이 인공지능(AI)을 동원해 일상적으로 최저가 경쟁을 펼치는 마당에 바이어들이 최저가를 정하는 게 무슨 의미가 있을까 싶다고 했다. 다만, 금융투자 업계에선 이런 결정이 이마트의 수익성 개선에 도움을 줄 것으로 본다. 한국투자증권이 추정한 이마트의 3분기 별도기준 영업이익은 990억원으로 전년 동기(1050억원)보다 5.7% 적다."
#text = "환율이란 양국 통화의 교환비율이다. 양국의 환율은 양국 상호간에 통화나 물품이 교환되는 가격, 즉 통화간의 교환비율로서 무역수지를 결정하는 가장 중요한 변수중의 하나이다. 일반적으로 통화의 가치는 그 통화가 가지는 구매력에 의하여 표현되는데, 1국 통화의 외국에서의 구매력은 외화와 교환됨으로 써 실현되기 때문에 1국 통화와 외국 통화의 교환비율로서의 환율은 1국 통화의 대외가치를 나타내는 것으로서 중요시된다. 한국의 경우는 달러나 마르크와 같은 외화를 원화와 교환할 때의 비율, 즉 외화를 원화로 매매할 때의 가격이라고 할 수 있다. 오늘날 외국과의 대차결제는 외국환에 의하여, 외국환은행이 외국과의 사이에서 지급을 필요로 하는 자에게 외국환어음이나 기타 외화채권(外貨債權)을 매매하는 형태로 이루어지고 있다. 따라서 실제적으로는 외국환시세, 즉 환율은 외국환은행이 이러한 외화 채권을 매매할 때의 가격으로 기능하고 있다. 환율은 일반상품의 가격형성과정과 같이 원칙적으로는 외화에 대한 수요와 공급의 관계에 따라서 변동한다. 그러나 이것이 늘 대폭으로 변동하면 무역이나 자본의 대차 등 국제거래에 지장이 많아진다. 일반적으로 환율이 하락(원화절상)하면 수출은 줄고 수입은 늘어나 경상수지가 나빠지고 우리나라와 같이 대외의존도가 높은 나라에서는 경제 성장률도 낮아지게 되나 한편으로는 수입물가가 떨어져 국내물가가 하락하고 기업의 외채상환능력이 줄어드는 등 밝은 면도 있다. 반면 환율이 상승(원화절하)하면 수출이 늘어나고 수입이 줄어드는 요인으로 작용하여 경상수지가 개선되지만 수입물가가 올라 국내물가 또한 상승하는 부작용을 초래할 수 있다. 본 논문에서는 원화의 달러 환율을 기준 환율로 여러 거시경제 변수 중 독립변수를 설정하고 모형을 예측한 뒤 다중공선성, 자기상관 분석을 통해 분석에 적합한 변수를 찾아내 보다 정교한 모형을 도출해 내고 다중회귀분석과 벡터자기회귀분석을 통해 환율에 영향을 미치는 요인에 대해 분석할 것이다. 환율이 변동하는 이유는 무엇일까? 환율은 기본적으로 개인, 기업, 금융기관 등의 외환시장 참가자들이 수출과 수입 등 외국과의 거래에 따라 외환시장에서 어느 정도의 외환을 필요로 하고 어느 정도의 외환을 외환시장에 공급하느냐에 따라 그 수준이 결정된다. 따라서 환율변동에 영향을 미치는 요인에는 물가, 경제성장률, 국내외 금리수준, 통화량, 국제수지, 국제유가 등 원자재 가격, 주식시장, 채권시장 등의 기초적 요인과 시장분위기, 통화당국의 외환시장 개입, 시장의 순환적 국면, 새로운 뉴스 등의 기술적 요인이 있다. 본 연구는 거시경제변수들이 환율에 어떠한 영향을 미치는지에 대하여 분석하기 위해 다중공선성과 자기상관을 통해 변수의 적합성을 도출해 낸 후에 환율과의 연관성을 알아본다. 분석에 사용된 자료는 대미 달러 환율, 종합주가지수(KOSPI종가), S&P 500지수, 금리(CD 유통수익률), 경상수지, 자본수지, 현금(M2), 물가상승률과 GDP Deflator 등의 거시경제 요인들이며 분석기간은 1999년 1월부터 2009년 12월까지로써 월별 데이터를 사용하여 분석하였다. 본 논문의 구성은 다음과 같다. 먼저 제 1 장 서론에서는 연구배경과 목적, 연구 방법을 소개하고 제 2 장에서는 한국의 환율제도와 한국 환율의 변화 추이를, 제 3 장에서는 선행연구, 제 4 장에서는 변수의 선정과 모형 예측 및 검정, 모형의 설정, 벡터자기회귀모형 등 실증분석에 관해 설명한다. 제 5 장에서는 결과를 도출해내고 한계점을 제시한다."
kw_extractor = yake.KeywordExtractor(lan='ko', top=100, COpy=True, Usage=False)
#DataCore(text=text, stopword_set=[], windowsSize=1, n=3)
keywords = kw_extractor.extract_keywords(text)

for kw in keywords[:50]:
 	print(kw)

('이마트가', 0.030938084739957763)
('최저가', 0.0384176636573153)
('가격을', 0.04971109560056476)
('인플레이션 방어를 명분으로', 0.0935546245162249)
('생활필수품 최저가 책정', 0.12369537448814878)
('유통업계에서', 0.12387193181652603)
('상품의', 0.12404081002272319)
('실행했던 연말까지 생활필수품', 0.12406202953806993)
('e커머스의', 0.15988225962558628)
('종료했다', 0.1608502213296334)
('납품업체들의', 0.1608502213296334)
('경쟁', 0.16185763602193753)
('부담은', 0.16197526706892)
('판매를', 0.16663727498427555)
('여건이 조성된', 0.19263795155067612)
('환율까지 급등해', 0.19438447958500288)
('명분으로 야심', 0.19664412833198075)
('야심 차게 실행했던', 0.19664412833198075)
('책정 전략을', 0.19664412833198075)
('바이어는', 0.2252459674775641)
('유명무실해진', 0.2267188728480907)
('정해져', 0.22946885855170412)
('소비자들의', 0.23102411337789755)
('가운데', 0.23102411337789755)
('낮추거나', 0.2314515654070294)
('주요', 0.23189661845715634)
('관계자는', 0.23284466246599653)
('업계', 0.23443433485679932)
('수준을', 0.23443433485679932)
('커졌다', 0.23627046931690832)
('행사하기', 0.2376634038559)
('부상', 0.23841926089170165)
('결정을', 0.23888

## 현재 pke_u_i9 모델 

In [23]:
import pke_u_i9.pke_u_i9 as pke_u_i9
keywords=pke_u_i9.extract_keywords(text)
for kw in keywords[:50]:
    print(kw)

('영업이익', 0.06440195756531672)
('충당금', 0.11799940203498949)
('반도체', 0.16003080649790893)
('글로벌', 0.2295259208160184)
('전기차', 0.2332941552852077)
('친환경차', 0.23474596385544866)
('서유럽', 0.23899857336621508)
('현대차', 0.26693935944979197)
('고수익', 0.3721143125395491)
('스포티지', 0.3722256568291371)
('카렌스', 0.37654989980676223)
('러시아', 0.3772684488887667)
('gdi', 0.4298899582412426)
('suv', 0.48798660083943757)
('일회성', 0.6712297803540104)
('경영실적', 0.7144514764728837)
('컨퍼런스', 0.7144514764728837)
('매출액', 0.7144514764728837)
('판매대수', 0.7720189202293004)
('당기순이익', 0.7848116167118221)
('경상이익', 0.8016305203392835)
('관계자', 0.8063252546590616)
('경쟁력', 0.8063252546590616)
('가운데', 0.8063252546590616)
('품질비용', 0.8063252546590616)
('하이브리드', 0.8156598544111111)
('지난해', 0.8219962349379091)
('고금리', 0.8233324434891817)
('인도공장', 0.8268835871081172)
('대부분', 0.8268835871081172)


In [3]:
komoran = Komoran()

In [4]:
STRTexT = text 
LISTText = ["현대차에 이어 기아도 올해 3분기(7~9월)에 사상 최대 분기 매출을 기록했다.",
            "그러나 세타 GDI 엔진 리콜 관련 1조5442억원의 대규모 충당금 반영으로 기아의 3분기 영업이익은 전년 동기 대비 42.1% 감소한 7682억원에 그쳤다.",
            "다만 일회성 비용인 리콜 충당금을 제외하면 사실상 영업이익이 2조원을 넘어서는 양호한 흐름을 이어갔다.",
            "◇3분기 매출은 역대 최대…1.5조 충당금 반영 영업이익은 42%↓ 기아는 25일 '2022년 3분기 경영실적 컨퍼런스 콜'을 통해 연결 기준 매출액이 23조1616억원으로 전년 동기 대비 30.5% 늘었다고 밝혔다.",
            "이는 지난 2분기에 기록한 역대 최대 분기 매출 21조8760억원을 뛰어넘은 것이다.",
            "영업이익은 7682억원으로 전년 동기 대비 42.1% 줄었다.",
            "지난 2분기와 비교하면 65.6%나 감소했다.",
            "지난 2분기에는 2조2340억원으로 역대 최대 분기 영업이익을 기록했었다.",
            "판매대수 증가와 고부가가치 차종 중심의 믹스 개선, 우호적 환율 효과에도 불구하고 세타 GDi 엔진 관련 충당금 1조5442억원이 반영돼 영업이익이 1조원대 밑으로 떨어졌다.",
            "영업이익률도 3.3%로 4.2%p 떨어졌다.",
            "3분기 경상이익은 7300억원, 당기순이익은 4589억원을 기록했다.",
            "기아 관계자는 “부품 수급 개선으로 판매가 증가했고, 높은 상품 경쟁력을 바탕으로 한 고수익 차량 중심의 판매로 수익 구조 개선이 지속된 가운데 우호적 환율 영향도 강하게 작용했으나, 엔진 품질비용이 크게 반영된 결과 영업이익 감소를 피할 수 없었다”고 밝혔다.",
            "이어 “다만 전 차종과 전 지역에 걸친 강한 수요는 지속되고 있는 만큼, 4분기에는 반도체 등 부품 수급 개선과 연계한 공급 확대를 통해 판매 회복과 수익성 강화가 이어질 것으로 기대한다”고 덧붙였다.",
            "◇3분기 글로벌 판매 75만2104대 판매…전년比 9.9%↑ 기아는 3분기 글로벌 시장에서 총 75만2104대를 판매했다고 밝혔다.",
            "글로벌 차량용 반도체 및 기타 부품의 공급난 완화로 전년 동기 대비 9.9% 늘었다.",
            "국내 시장에서는 전년 동기 대비 6.2% 증가한 13만 2768대를 판매했다.",
            "이는 EV6·신형 니로 등 신차 효과와 반도체 공급난 완화 등으로 SUV 주요 모델들의 대기 수요가 일부 해소된데 따른 것이다.",
            "해외 시장에서는 10.7% 증가한 61만9336대를 판매했다.",
            "러시아-우크라이나 전쟁으로 러시아 판매 중단의 영향이 본격화됐지만 수익성이 높은 타 지역으로의 물량 전환과 인도공장의 3교대 전환, 카렌스와 신형 스포티지 등 신차 효과로 대부분 지역에서 판매가 증가했다.",
            "기아의 3분기 친환경차 판매는 전년 동기 대비 46.8% 증가한 12만3000대를 기록했다.",
            "유형별로 △전기차가 4만대(전년 대비 34.3%↑) △하이브리드가 6만2000대(67.0%↑) △플러그인 하이브리드가 2만1000대(24.1%↑)였다.",
            "전체 판매 차종 중 친환경차 비중도 전년 대비 5.6%p 오른 16.8%에 달했다.",
            "국내는 12.3%, 서유럽은 11.%, 미국은 3.2%를 기록하는 등 전년 동기 대비 전기차 판매 비중이 크게 올랐다.",
            "전기차 판매가 특정 지역에 집중되던 것에서 벗어나 시장도 다변화됐다.",
            "지난해 3분기 서유럽 비중이 52.9%에 달했으나 올해에는 국내가 40.8%로 가장 높았고 서유럽 38.9%, 미국 14.6% 등 주요 시장에서 고른 판매를 기록했다.",
            "◇국제 정세·경제 상황 불안하지만…4분기 가시적 실적 개선 기대 기아는 국제 정세 불안에 따른 원자재 가격 변동성 심화, 고금리와 인플레이션으로 인한 구매 심리 위축 등 불안정한 대외 환경을 예의주시하면서도 4분기 가시적인 실적 개선을 기대했다.",
            "반도체 등 부품 수급 상황 개선과 연계해 공급을 최대한 늘림으로써 높은 대기 수요를 빠르게 해소하고, 친환경차와 고수익 RV 모델 중심으로 판매를 확대해 수익성 강화를 지속 추진한다는 계획이다.",
            "전 세계에서 인정받고 있는 전기차 EV6의 생산·판매를 확대하면서 미국에서 텔루라이드 상품성 개선 모델 및 신형 스포티지, 유럽에서 신형 니로, 인도에서 카렌스 등 시장별 핵심 차종 판매 본격화에 집중한다는 방침이다.",
            "아울러 제품 및 트림 믹스를 지속적으로 높이고, 개선된 브랜드 가치와 상품성에 부응하는 가격 정책을 이어가며 수익성도 극대화할 예정이다.",
            ]   

# new_text = edit_sentences(STRTexT)
# # print('원본 split_sentences :', new_text)
# total_value = []
# for x in range(len(new_text)):
#     total_value.append(' '.join(edit_josa(new_text[x])))

# print(total_value)
TUNNEDText = ['현대차 이어 기아 올해 3분기 7 9월 사상 최대 분기 매출 기록', '그러나 세타 GDI 엔진 리콜 관련 1조 대규모 충당금 반영 기아 3분기 영업이익 전년 동기 대비 42.1 감소 7682억 그치', '다만 일회 비용 리콜 충당금 제외 사실 영업이익 2조 넘어서 양호 흐름 잇', '◇ 매출 역대 최대 충당금 반영 영업이익 42% 기아 25일 2022년 3분기 경영실적 컨퍼런스 콜 통하 연결 기준 매출액 23조 전년 동기 대비 30.5 늘 밝히', '이 지나 2분기 기록 역대 최대 분기 매출 21조 뛰어넘 것', '영업이익 7682억 전년 동기 대비 42.1 줄', '지나 2분기 비교 65.6% 감소', '지나 2분기 2조 역대 최대 분기 영업이익 기록', '판매대수 증가 고부가가치 차종 중심 믹스 개선 우호 환율 효과 불구 세타 GDi 엔진 관련 충당금 1조 반영 영업이익 1조 밑 떨어지', '영업이익 3.3% 4.2% 떨어지', '3분기 경상이익 7300억 당기순이익 4589억 기록', '기아 관계자 부품 수급 개선 판매 증가 높 상품 경쟁력 바탕 한 고수익 차량 중심 판매 수익 구조 개선 지속 가운데 우호 환율 영향 강하 작용 엔진 품질비용 크 반영 결과 영업이익 감소 피하 수 없 밝히', '이어 다만 전 차종 전 지역 걸치 강하 수요 지속 있 만큼 4분기 반도체 등 부품 수급 개선 연계 공급 확대 통하 판매 회복 수익 강화 이어지 것 기대 덧붙이', '◇ 글로벌 판매 75만 판매 9.9% 기아 3분기 글로벌 시장 총 75만 판매 밝히', '글로벌 차량 반도체 및 기타 부품 공급 완화 전년 동기 대비 9.9 늘', '국내 시장 전년 동기 대비 6.2 증가 13만 2768대 판매', '이 EV 니 등 신차 효과 반도체 공급 완화 등 SUV 주요 모델 대기 수요 일부 해소 따르 것', '해외 시장 10.7 증가 61만 판매', '러시아 전쟁 러시아 판매 중단 영향 본격 수익 높 타 지역 물량 전환 인도공장 3교대 전환 카렌스 신형 스포티 등 신차 효과 대부분 지역 판매 증가', '기아 3분기 친환경차 판매 전년 동기 대비 46.8 증가 12만 기록', '유형 △ 4만 전년 대비 34.3% △ 6만 67.0% △ 하이브리드 2만 24.1%', '전체 판매 차종 중 친환경차 비중 전년 대비 5.6% 오르 16.8% 달하', '국내 12.3 서유럽 미국 3.2% 기록 등 전년 동기 대비 전기차 판매 비중 크 오르', '전기차 판매 특정 지역 집중 것 벗어나 시장 다변', '지난해 3분기 서유럽 비중 52.9% 달하 올해 국내 40.8% 가장 높 서유럽 38.9 미국 14.6 등 주요 시장 고르 판매 기록', '◇ 정세 상황 불안 가시 실적 개선 기대 기아 국제 정세 불안 따르 원자재 가격 변동 심화 고금리 인플레이션 인하 구매 심리 위축 등 불안정 대외 환경 예의주시 4분기 가시 실적 개선 기대', '반도체 등 부품 수급 상황 개선 연계 공급 최대한 늘리 높 대기 수요 빠르 해소 친환경차 고수익 RV 모델 중심 판매 확대 수익 강화 지속 추진 계획', '전 세계 인정받 있 전기차 EV 생산 확대 미국 텔루 상품 개선 모델 및 신형 스포티 유럽 신형 니 인도 카렌스 등 시장 핵심 차종 판매 본격 집중 방침', '아울러 제품 및 트림 믹스 지속 높이 개선 브랜드 가치 상품 부응 가격 정책 이어 수익 극대 예정']

## 빈도수 기반 명사 추출

In [5]:
# STRTexT에서 명사를 추출, 명사의 수를 count로 선언
noun = komoran.nouns(STRTexT)
count = Counter(noun)

# 명사 빈도 카운트q
noun_list = count.most_common(20)
for v in noun_list:
    if len(v[0]) == 1:
        pass
    else:
        print("Komoran 기준 : " + str(v))

Komoran 기준 : ('판매', 19)
Komoran 기준 : ('분기', 18)
Komoran 기준 : ('전년', 10)
Komoran 기준 : ('이익', 9)
Komoran 기준 : ('대비', 9)
Komoran 기준 : ('개선', 9)
Komoran 기준 : ('영업', 8)
Komoran 기준 : ('기아', 7)
Komoran 기준 : ('기록', 7)
Komoran 기준 : ('동기', 7)
Komoran 기준 : ('수익', 7)
Komoran 기준 : ('증가', 6)
Komoran 기준 : ('시장', 6)
Komoran 기준 : ('최대', 4)
Komoran 기준 : ('충당금', 4)
Komoran 기준 : ('반영', 4)


## TF.IDF 

In [6]:
vectorizer = TfidfVectorizer()
vectorizer.fit(TUNNEDText)
matrix = vectorizer.transform(TUNNEDText)

vocabulary_word_id = defaultdict(int)
    
for idx, token in enumerate(vectorizer.get_feature_names()):
    vocabulary_word_id[token] = idx
    
# 특징 추출 결과: {"token": value}
result = defaultdict(str)
    
for token in vectorizer.get_feature_names():
    result[token] = matrix[0, vocabulary_word_id[token]]
    
# 내림차순 (중요도 high) 기준 정렬
result = sorted(result.items(), key = lambda item: item[1], reverse = True)
pprint(result[:20])

[('9월', 0.36467446274117504),
 ('사상', 0.36467446274117504),
 ('현대차', 0.36467446274117504),
 ('올해', 0.3247983115456801),
 ('매출', 0.29650573201683855),
 ('분기', 0.29650573201683855),
 ('이어', 0.29650573201683855),
 ('최대', 0.2745603024038633),
 ('3분기', 0.22833700129250212),
 ('기록', 0.22833700129250212),
 ('기아', 0.22833700129250212),
 ('10', 0.0),
 ('12', 0.0),
 ('12만', 0.0),
 ('13만', 0.0),
 ('14', 0.0),
 ('16', 0.0),
 ('1조', 0.0),
 ('2022년', 0.0),
 ('21조', 0.0)]


c:\Users\BIG-LGY\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Text Summarization

In [14]:
# 원 예제는 영어를 입력 하면 번역 후 한국어로 출력되는 기능이 있음.
# 이번에는 한국어를 입력했기 때문에 해당 기능이 사용되지 않음. (영어 입력하면 한국어 나오는지는 실험 하지 않음.)

# 사용된 방식은 추상적 방법론을 사용해 만든 로직임. 추상적 방법론이란, 대상 문서의 의미를 해석하여 지능적으로 요약문을 만드는 방법으로, 완전히 새로운 문장을 생성함.
# 다른 방식으로는 대상 문서에서 중요한 문장을 식별하여 요약문을 만드는 방법인 추출식 방법론이 있음.

def summarize(text, per):
    nlp = spacy.load('ko_core_news_sm')
    doc = nlp(text)
    tokens = [token.text for token in doc]
    word_frequencies = {}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1
    max_frequency = max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word] = word_frequencies[word]/max_frequency
    sentence_tokens = [sent for sent in doc.sents]
    sentence_score = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_score.keys():
                    sentence_score[sent] = word_frequencies[word.text.lower()]
                else:
                    sentence_score [sent] += word_frequencies[word.text.lower()]

    select_length = int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_score, key=sentence_score.get)
    final_summary = [word.text for word in summary]
    summary = ''.join(final_summary)
    return summary

summarized_text = summarize(STRTexT, 0.1)
print('10% 요약 결과: ', summarized_text)

10% 요약 결과:  ◇3분기 매출은 역대 최대…1.5조 충당금 반영 영업이익은 42%↓ 기아는 25일 '2022년 3분기 경영실적 컨퍼런스 콜'을 통해 연결 기준 매출액이 23조1616억원으로 전년 동기 대비 30.5% 늘었다고 밝혔다.전 세계에서 인정받고 있는 전기차 EV6의 생산·판매를 확대하면서 미국에서 텔루라이드 상품성 개선 모델 및 신형 스포티지, 유럽에서 신형 니로, 인도에서 카렌스 등 시장별 핵심 차종 판매 본격화에 집중한다는 방침이다.


### * Text Summarization With Attention

In [15]:
# 해당 기능은 기계학습이 필요하기 때문에 후순위로 선정한다.

### * TextRank Based on Sentence Embedding

In [16]:
# 해당 기능은 기계학습이 필요하기 때문에 후순위로 선정한다.


# Glove를 활용한 

## RAKE

## WordRank

### * KR-WordRank

In [17]:
texts = TUNNEDText
# texts = LISTText
wordrank_extractor = KRWordRank(min_count=5, max_length=10)
keywords, rank, graph = wordrank_extractor.extract(texts, num_keywords=100)

def make_vocab_score(keywords, scaling=None):
    if scaling is None:
        scaling = lambda x:math.sqrt(x)
    return {word:scaling(rank) for word, rank in keywords.items()}

keywords = make_vocab_score(keywords)
print(keywords)

{'판매': 1.6568342303943329, '전년': 1.3343437243657996, '대비': 1.1938222184638514, '기록': 1.1625576798349897, '동기': 1.0767190566836524, '기아': 1.0693013192400256, '개선': 1.0677853006972664, '영업이익': 1.010704561676164, '3분': 0.9970076164823614, '증가': 0.8998219373682382, '시장': 0.8837040969523341, '최대': 0.6900709839984251, '수익': 0.5768966126840452}


### * sentence extraction

In [18]:
texts = LISTText
keywords, sents = summarize_with_sentences(texts, num_keywords=100, num_keysents=10)
print(sents)

['기아의 3분기 친환경차 판매는 전년 동기 대비 46.8% 증가한 12만3000대를 기록했다.', "◇3분기 매출은 역대 최대…1.5조 충당금 반영 영업이익은 42%↓ 기아는 25일 '2022년 3분기 경영실적 컨퍼런스 콜'을 통해 연결 기준 매출액이 23조1616억원으로 전년 동기 대비 30.5% 늘었다고 밝혔다.", '◇3분기 글로벌 판매 75만2104대 판매…전년比 9.9%↑ 기아는 3분기 글로벌 시장에서 총 75만2104대를 판매했다고 밝혔다.', '기아 관계자는 “부품 수급 개선으로 판매가 증가했고, 높은 상품 경쟁력을 바탕으로 한 고수익 차량 중심의 판매로 수익 구조 개선이 지속된 가운데 우호적 환율 영향도 강하게 작용했으나, 엔진 품질비용이 크게 반영된 결과 영업이익 감소를 피할 수 없었다”고 밝혔다.', '전체 판매 차종 중 친환경차 비중도 전년 대비 5.6%p 오른 16.8%에 달했다.', '지난해 3분기 서유럽 비중이 52.9%에 달했으나 올해에는 국내가 40.8%로 가장 높았고 서유럽 38.9%, 미국 14.6% 등 주요 시장에서 고른 판매를 기록했다.', '이어 “다만 전 차종과 전 지역에 걸친 강한 수요는 지속되고 있는 만큼, 4분기에는 반도체 등 부품 수급 개선과 연계한 공급 확대를 통해 판매 회복과 수익성 강화가 이어질 것으로 기대한다”고 덧붙였다.', '현대차에 이어 기아도 올해 3분기(7~9월)에 사상 최대 분기 매출을 기록했다.', '전 세계에서 인정받고 있는 전기차 EV6의 생산·판매를 확대하면서 미국에서 텔루라이드 상품성 개선 모델 및 신형 스포티지, 유럽에서 신형 니로, 인도에서 카렌스 등 시장별 핵심 차종 판매 본격화에 집중한다는 방침이다.', '글로벌 차량용 반도체 및 기타 부품의 공급난 완화로 전년 동기 대비 9.9% 늘었다.']


## TextRank 

In [19]:
class RawSentence:
    def __init__(self, textIter):
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s
 
class RawSentenceReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s
 
class RawTagger:
    def __init__(self, textIter, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)
 
class RawTaggerReader:
    def __init__(self, filepath, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)
 
class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.window = kargs.get('window', 5)
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}
        self.dictNear = {}
        self.nTotal = 0
 
 
    def load(self, sentenceIter, wordFilter = None):
        def insertPair(a, b):
            if a > b: a, b = b, a
            elif a == b: return
            self.dictBiCount[a, b] = self.dictBiCount.get((a, b), 0) + 1
 
        def insertNearPair(a, b):
            self.dictNear[a, b] = self.dictNear.get((a, b), 0) + 1
 
        for sent in sentenceIter:
            for i, word in enumerate(sent):
                if wordFilter and not wordFilter(word): continue
                self.dictCount[word] = self.dictCount.get(word, 0) + 1
                self.nTotal += 1
                if i - 1 >= 0 and (not wordFilter or wordFilter(sent[i-1])): insertNearPair(sent[i-1], word)
                if i + 1 < len(sent) and (not wordFilter or wordFilter(sent[i+1])): insertNearPair(word, sent[i+1])
                for j in range(i+1, min(i+self.window+1, len(sent))):
                    if wordFilter and not wordFilter(sent[j]): continue
                    if sent[j] != word: insertPair(word, sent[j])
 
    def loadSents(self, sentenceIter, tokenizer = None):
        import math
        def similarity(a, b):
            n = len(a.intersection(b))
            return n / float(len(a) + len(b) - n) / (math.log(len(a)+1) * math.log(len(b)+1))
 
        if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
        sentSet = []
        for sent in filter(None, sentenceIter):
            if type(sent) == str:
                if tokenizer: s = set(filter(None, tokenizer(sent)))
                else: s = set(filter(None, rgxSplitter.split(sent)))
            else: s = set(sent)
            if len(s) < 2: continue
            self.dictCount[len(self.dictCount)] = sent
            sentSet.append(s)
 
        for i in range(len(self.dictCount)):
            for j in range(i+1, len(self.dictCount)):
                s = similarity(sentSet[i], sentSet[j])
                if s < self.threshold: continue
                self.dictBiCount[i, j] = s
 
    def getPMI(self, a, b):
        import math
        co = self.dictNear.get((a, b), 0)
        if not co: return None
        return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])
 
    def getI(self, a):
        import math
        if a not in self.dictCount: return None
        return math.log(self.nTotal / self.dictCount[a])
 
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
 
    def extract(self, ratio = 0.1):
        ranks = self.rank()
        cand = sorted(ranks, key=ranks.get, reverse=True)[:int(len(ranks) * ratio)]
        pairness = {}
        startOf = {}
        tuples = {}
        for k in cand:
            tuples[(k,)] = self.getI(k) * ranks[k]
            for l in cand:
                if k == l: continue
                pmi = self.getPMI(k, l)
                if pmi: pairness[k, l] = pmi
 
        for (k, l) in sorted(pairness, key=pairness.get, reverse=True):
            print(k[0], l[0], pairness[k, l])
            if k not in startOf: startOf[k] = (k, l)
 
        for (k, l), v in pairness.items():
            pmis = v
            rs = ranks[k] * ranks[l]
            path = (k, l)
            tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
            last = l
            while last in startOf and len(path) < 7:
                if last in path: break
                pmis += pairness[startOf[last]]
                last = startOf[last][1]
                rs *= ranks[last]
                path += (last,)
                tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
 
        used = set()
        both = {}
        for k in sorted(tuples, key=tuples.get, reverse=True):
            if used.intersection(set(k)): continue
            both[k] = tuples[k]
            for w in k: used.add(w)
 
        #for k in cand:
        #    if k not in used or True: both[k] = ranks[k] * self.getI(k)
 
        return both
 
    def summarize(self, ratio = 0.333):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)[:int(len(r)*ratio)]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))

### * 문장 요약

In [20]:
tr = TextRank()
stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])
tr.loadSents(RawSentenceReader('test.txt'), lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), komoran.pos(sent)))
# print('Build...')
tr.build()
ranks = tr.rank()
for k in sorted(ranks, key=ranks.get, reverse=True)[:100]:
    print("\t".join([str(k), str(ranks[k]), str(tr.dictCount[k])]))
print(tr.summarize(0.2))

1429	0.0018812609738587198	 2021년 1·4분기는 2.43%, 올해 2·4분기는 0.55%다.
801	0.0016846434465376731	올해 런칭 예정인 브랜드는 두 가지가 더 있습니다.
393	0.0016155314543980512	 한국은행에 따르면 올해 6월 말 기준 금융권 PF대출 잔액은 112조2000억원입니다.
783	0.0015612503540812857	 이에 올해 캉골의 예상 매출액은 전년 대비 약 19.5% 증가한 ?
1413	0.0015287112365825594	 지난해 말까지 증권사들의 수익률이 타 금융권보다 월등히 높았다.
1891	0.0015086648045184547	 국내 시장에서는 전년 동기 대비 6.2% 증가한 13만 2768대를 판매했다.
785	0.0014560830183853865	 [캉골 브랜드 이미지] 캉골키즈는 올해 2분기 매출액 108억원을 기록했고 올해 예상 매출액은 ?
1807	0.0014183258358390128	 CJ ENM의 올해 2분기 영업이익은 556억원으로 전년 동기 대비 35.2% 감소했다.
1492	0.0013918673775085654	 국내 기업들의 3·4분기 실적 발표도 이어진다.
555	0.0013831774220597086	 시장에선 이번 사태를 꽤 심각하게 보고 있다.
1596	0.0013820421760259412	 쏘렌토는 올해 1~9월 국내 시장에서 4만9726대가 팔려 전체 브랜드 가운데 1위를 차지했다.
1882	0.0013752654618197704	 지난 2분기와 비교하면 65.6%나 감소했다.
1889	0.0013556330174771761	 ◇3분기 글로벌 판매 75만2104대 판매…전년比 9.9%↑ 기아는 3분기 글로벌 시장에서 총 75만2104대를 판매했다고 밝혔다.
845	0.001344892526475769	 검찰의 수사 방향이 감사원과 같다면, 검찰도 문 전 대통령에 대한 조사가 필요하다고 결론지을 수도 있다.
125	0.00134

### * 키워드 추출

In [34]:
tr = TextRank(window=5, coef=1)
# print('Load...')
stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV'), ('없', 'VV') ])
tr.load(RawTaggerReader('test.txt', tagger=komoran), lambda w: w not in stopword and (w[1] in ('NNG', 'NNP', 'VV', 'VA')))
# print('Build...')
tr.build()
kw = tr.extract(0.1)
for k in sorted(kw, key=kw.get, reverse=True):
    print("%s\t%g" % (k, kw[k]))

소상 공인 8.460411177317253
해 보 8.437938321465195
아이오 닉 8.373399800327624
전용 헬기 8.268039284669797
우크라이나 전쟁 8.25561676467124
세액 공제 8.237267626003042
성남 도시 8.159825700029565
부원 장 8.12393894069604
전당 대회 8.085717727875842
도시 개발 8.085717727875842
김용 민주 8.085717727875842
비상 경제 8.085717727875842
코로나 19 7.967934692219458
퇴직 연금 7.964468484242973
유동 규 7.921414676584566
스타트 업 7.9138674709491825
보 카 7.834403299594936
여 심위 7.8137840123922
인천 공항 7.8137840123922
비트코인 채굴 7.744791140905249
정당 지지 7.7315459141552285
뉴미디어 비서관 7.6802526197676775
개발 공사 7.668418162120675
신라 스테이 7.661903481099481
연구원 부원 7.659633332564942
공무원 피격 7.659633332564942
피격 사건 7.659633332564942
신라 젠 7.621412119744744
지급 보증 7.5624695841112946
방통 위 7.5395852902777065
여론 조사 7.482426876437758
스마트 주차 7.482426876437758
영어 학습 7.479581924305527
우주 항공 7.457109068453468
전년 동기 7.444686548454911
매매 가격 7.392570547315897
양국 관계 7.361798888649143
서해 공무원 7.3437803831464645
순 매수 7.3437803831464645
패권 경쟁 7.317347126078309
민간 사업자 7.31252783964236
현지 시간 7.312

## TopicRank 

## TopicalPageRank 

## PositionRank 

## MultipartiteRank  

## ExpandRank 

## KEA 

## KeyBERT 

### * Korean KeyBERT

In [26]:
from konlpy.tag import Okt
okt = Okt()
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [27]:

tokenized_doc = okt.pos(STRTexT)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

n_gram_range = (1, 3)
count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수: ', len(candidates))
print('trigram 5개 출력: ', candidates[:5])


doc_embedding = model.encode([STRTexT])
candidate_embeddings = model.encode(candidates)

top_n = 10
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

품사 태깅 10개만 출력 : [('현대차', 'Noun'), ('에', 'Josa'), ('이어', 'Verb'), ('기아', 'Noun'), ('도', 'Josa'), ('올해', 'Noun'), ('3분', 'Number'), ('기', 'Foreign'), ('(', 'Punctuation'), ('7~9월', 'Number')]
명사 추출 : 현대차 기아 올해 사상 최대 분기 매출 기록 세타 엔진 리콜 관련 대규모 충당금 반영 기아 영업 이익 년 동기 대비 감소 다만 회 비용 리콜 충당금 제외 사실 영업 이익 양호 흐름 매출 역대 최대 충당금 반영 영업 이익 기아 경영 실적 컨퍼런스 콜 통해 연결 기준 매출 액 년 동기 대비 지난 기록 역대 최대 분기 매출 것 영업 이익 년 동기 대비 지난 비교 나 감소 지난 역대 최대 분기 영업 이익 기록 판매 대수 증가 부가가치 차종 중심 믹스 개선 우호 환율 효과 불구 세타 엔진 관련 충당금 반영 영업 이익 밑 영업 이익 률 로 경상 이익 순이익 기록 기아 관계자 부품 수급 개선 판매 증가 상품 경쟁력 바탕 수익 차량 중심 판매 수익 구조 개선 지속 가운데 우호 환율 영향 작용 엔진 품질 비용 크게 반영 결과 영업 이익 감소 피 수 고 다만 전 차종 전 지역 수요 지속 만큼 반도체 등 부품 수급 개선 계 공급 확대 통해 판매 회복 수익 강화 것 고 글로벌 판매 판매 년 기아 글로벌 시장 총 판매 글로벌 차량 용 반도체 및 기타 부품 공급 완화 년 동기 대비 국내 시장 년 동기 대비 증가 판매 신형 니 등 신차 효과 반도체 공급 완화 등 주요 모델 대기 수요 일부 해소 것 해외 시장 증가 판매 러시아 우크라이나 전쟁 러시아 판매 중단 영향 본격 수익 타 지역 물량 전환 인도 공장 교대 전환 카 렌스 신형 스포티지 등 신차 효과 대부분 지역 판매 증가 기아 친환 경차 판매 년 동기 대비 증가 기록 유형 별로 전기차 년 대비 하이브리드 플러그인 하이브리드 전체 판매 차종 중 친환 경차 비중 년 대비 오른 달 국내 서유

### * Max Sum Similarity : 
데이터 쌍 사이의 최대 합 거리는 데이터 쌍 간의 거리가 최대화되는 데이터 쌍으로 정의됨.
후보 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화하고자 하는 로직임.

In [28]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]


print(max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=6, nr_candidates=10))
print(max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=6, nr_candidates=30))  # nr_candidates가 높을수록 다양한 키워드 추출

['최대 분기 영업', '현대차 기아', '신차 효과 대부분', '순이익 기록 기아', '수익 성도 극대', '엔진 품질 비용']
['지난 역대 최대', '작용 엔진 품질', '이익 순이익 기록', '현대차 기아', '신차 효과 대부분', '순이익 기록 기아']


### * Maximal Marginal Relevance : 텍스트 요약 작업에서 중복을 최소화하고 결과의 다양성을 극대화하는 로직. 문서와 가장 유사한 키워드/키프레이즈를 선택한 후 문서와 유사하고 이미 선택된 키워드/키프레이즈와 유사하지 않은 새로운 후보를 반복적으로 선택함

In [30]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]


print(mmr(doc_embedding, candidate_embeddings, candidates, top_n=10, diversity=0.2)) # 상대적으로 낮은 diversity값을 설정하면, 기존의 코사인 유사도만 사용한 것과 매우 유사함
print(mmr(doc_embedding, candidate_embeddings, candidates, top_n=10, diversity=0.7)) # 상대적으로 높은 diversity값을 설정하면 다양한 키워드를 추출함

['현대차 기아 올해', '수익 성도 극대', '엔진 품질 비용', '신차 효과 대부분', '순이익 기록 기아', '최대 분기 매출', '기록 세타 엔진', '현대차 기아', '대규모 충당금 반영', '양호 흐름 매출']
['현대차 기아 올해', '기타 부품', '러시아', '영업 이익 동기', '반도체', '신형 인도 렌스', '방침 제품 트림', '전쟁', '가장 서유럽 미국', '심화 금리 인플레이션']
